In [25]:
import os
from unstructured.partition.auto import partition

pdf_path = os.path.expanduser("~/pdf_to_rag/assets/RAG_em_CienciaDeDados.pdf")
elements = partition(pdf_path)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/ryan.oliveira/pdf_to_rag/venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/ryan.oliveira/pdf_to_rag/venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/ryan.oliveira/pdf_to_rag/venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
  

In [26]:
paragraphs = [el for el in elements if el.category == "NarrativeText"]
for i, paragraph in enumerate(paragraphs[:5]):
    print(f"Paragraph {i+1}:\n{paragraph.text}\n")

Paragraph 1:
Nesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.

Paragraph 2:
Vamos começar pelo começo: O que é RAG? RAG (Geração Aumentada por Recuperação) é uma técnica de inteligência artificial que combina o poder de recuperação de informações relevantes em grandes bases de dados com a geração de texto. Basicamente, ele funciona trazendo o melhor de dois mundos: a precisão de dados históricos e contextuais e a criatividade de modelos de linguagem natural, principalmente LLM ou Modelos de Linguagem Grandes como o GPT. Isso permite que sistemas RAG façam recomendações ou gerem respostas muito mais contextualizadas e detalhadas do que um modelo isolado de IA.

Paragraph 3:
Por que o RAG é importante para a Ciência de Dados e Negócios? A principal vantagem do RAG é que ele oferece respostas mais precisas e contextualizadas ao extrair informações relevant

In [27]:
print(vars(paragraphs[0]))

{'text': 'Nesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.', 'embeddings': None, '_element_id': '0a69dbb5a1ad8d51199d3a77641ef2f5', 'metadata': <unstructured.documents.elements.ElementMetadata object at 0x1403c1010>}


In [ ]:
vars_paragraph = {
    'text': 'Nesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.',
    'embeddings': None,
    '_element_id': '0a69dbb5a1ad8d51199d3a77641ef2f5',
    'metadata': "<unstructured.documents.elements.ElementMetadata object at 0x1403c1010>"
}

In [29]:
metadata = paragraphs[0].metadata
print(vars(metadata))

{'coordinates': CoordinatesMetadata(points=((36.0, 94.24400000000003), (36.0, 123.28399999999999), (561.82, 123.28399999999999), (561.82, 94.24400000000003)), system=<unstructured.documents.coordinates.PixelSpace object at 0x1402b4e90>), 'file_directory': '/Users/ryan.oliveira/pdf_to_rag/assets', 'filename': 'RAG_em_CienciaDeDados.pdf', 'last_modified': '2025-08-06T08:22:22', 'links': [], 'page_number': 1, 'languages': ['por'], '_known_field_names': frozenset({'detection_class_prob', 'header_footer_type', 'text_as_html', 'filetype', 'page_name', 'links', 'image_path', 'image_mime_type', 'link_start_indexes', 'sent_from', 'link_urls', 'cc_recipient', 'subject', 'signature', 'is_continuation', 'filename', 'emphasized_text_contents', 'url', 'data_source', 'sent_to', 'last_modified', 'email_message_id', 'parent_id', 'languages', 'coordinates', 'category_depth', 'emphasized_text_tags', 'image_url', 'key_value_pairs', 'table_as_cells', 'file_directory', 'page_number', 'image_base64', 'detect

In [ ]:
metadata_paragraph = {
    'coordinates': "CoordinatesMetadata(points=((36.0, 94.24400000000003), (36.0, 123.28399999999999), (561.82, 123.28399999999999), (561.82, 94.24400000000003)), system=<unstructured.documents.coordinates.PixelSpace object at 0x1402b4e90>)",
    'file_directory': '/Users/ryan.oliveira/pdf_to_rag/assets',
    'filename': 'RAG_em_CienciaDeDados.pdf',
    'last_modified': '2025-08-06T08:22:22',
    'links': [],
    'page_number': 1,
    'languages': ['por'],
    '_known_field_names': frozenset({'detection_class_prob', 'header_footer_type', 'text_as_html', 'filetype', 'page_name', 'links', 'image_path', 'image_mime_type', 'link_start_indexes', 'sent_from', 'link_urls', 'cc_recipient', 'subject', 'signature', 'is_continuation', 'filename', 'emphasized_text_contents', 'url', 'data_source', 'sent_to', 'last_modified', 'email_message_id', 'parent_id', 'languages', 'coordinates', 'category_depth', 'emphasized_text_tags', 'image_url', 'key_value_pairs', 'table_as_cells', 'file_directory', 'page_number', 'image_base64', 'detection_origin', 'orig_elements', 'attached_to_filename', 'link_texts', 'bcc_recipient'}),
    'filetype': 'application/pdf',
    'parent_id': '01ca41a6dcf3acc477735e9f013083f2'
}

In [30]:
from pydantic import BaseModel

class MetadataModel(BaseModel):
    coordinates: str
    file_directory: str
    filename: str
    last_modified: str
    links: list
    page_number: int
    languages: list
    _known_field_names: frozenset
    filetype: str
    parent_id: str

class ParagraphModel(BaseModel):
    text: str
    embeddings: None
    _element_id: str
    metadata: MetadataModel

In [34]:
paragraph_models = []
for paragraph in paragraphs: 
    metadata_dict = vars(paragraph.metadata)

    if not isinstance(metadata_dict['coordinates'], str):
        metadata_dict['coordinates'] = str(metadata_dict['coordinates'])
    
    metadata_model = MetadataModel(**metadata_dict)
    
    paragraph_model = ParagraphModel(
        text=paragraph.text,
        embeddings=getattr(paragraph, "embeddings", None),
        _element_id=getattr(paragraph, "_element_id", ""),
        metadata=metadata_model
    )

    paragraph_models.append(paragraph_model)

In [40]:
print(f"Text: {paragraph_models[0].text}")
print(f"Page Number: {paragraph_models[0].metadata.page_number}")
print(f"File Directory: {paragraph_models[0].metadata.file_directory}")
print(f"Filename: {paragraph_models[0].metadata.filename}")
print(f"Languages: {paragraph_models[0].metadata.languages}")

Text: Nesta edição, vamos explorar o conceito de RAG (Retrieval Augmented Generation) e como essa tecnologia pode transformar a forma como manipulamos dados e tomamos decisões nos negócios.
Page Number: 1
File Directory: /Users/ryan.oliveira/pdf_to_rag/assets
Filename: RAG_em_CienciaDeDados.pdf
Languages: ['por']
